# CS215: Intro to Algorithms

In [1]:
import math
import scipy
import scipy.special

## A Social Network Magic Trick

* A social network is connections between individuals that capture relationships between them.
* Algorithms are how we organize computations to solve problems.

A graph is collection of _nodes_ connected by _edges_. We can talk about the _degree_ of a node as the number of edges connecting to it.

See https://en.wikipedia.org/wiki/Graph_theory and https://en.wikipedia.org/wiki/Eulerian_path

An _Eulerian path_ is a path through a network that traverses each edge _once_, beginning at some node and ending at some node. A graph with an Eulerian path may have more than one Eulerian path.

If a graph is connected and has exactly two odd degree nodes, the Eulerian path starts and/or ends in those nodes. Nodes of even degree cannot be the start or end nodes of the Eulerian path.

Can a graph with only even degree nodes have an Eulerian path?

* No?
* Yes, but only certain graphs.
* **Yes, all such graphs do.**
* Yes, all graphs have an Eulerian path.

According to wikipedia, an undirected, connected graph has an Eulerian path iff it has either 0 or 2 vertices of odd degree. If it has 0 vertices of odd degree, the Eulerian path is an Eulerian circuit.

An _Eulerian tour_ is an Eulerian path that starts and ends at the same node.

Algorithms are cool

* really useful (performance)
* really clever

Practical - how to make a program fly?

* careful program design
* tweaking loops
* **good algorithm design**

Fake Python algorithm to think about algorithms

    def algorithm_development(problem_spec):
        correct = False
        while not correct or not fast_enough(running_time):
            algorithm = devise_algorithm(problem_spec)
            correct = analyze_correctness(algorithm)
            running_time = analyze_efficiency(algorithm)
        return algorithm

There is a heavy emphasis on mathematics in algorithm design

1. formalize what you are doing
2. analyze correctness
3. analyze efficiency

What does efficiency mean? Clock time? Memory? Power consumption?

In [2]:
def naive(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        z = z + y
        x = x - 1
    return z

In [3]:
naive(5, 2)

10

In [4]:
naive(2, 5)

10

In [5]:
naive(3, 3)

9

How do we show `naive` is "correct"? Observe: before or after the while loop, `a * b = x * y + z`. _Inductive step_: if `ab = xy + z` _before_ the loop, then `ab = x'y' + z'` _after_ the loop. Here, `x' = x - 1`, `y' = y`, `z' = z + y`, so do some algebra:

\begin{equation}
\begin{split}
x'y' + z' =&~ (x - 1)y + (z + y) \\
=&~ xy - y + z + y \\
=&~ xy + z
\end{split}
\end{equation}

So, while the code is running, `ab = xy + z` is always true. At the end, `x = 0`, so `z = ab`. We return `z`, so, therefore, the algorithm is multiplying `a` and `b`.

#### Running Time of `naive(a, b)`

`naive(a, b)` gets slower as `a` and `b` get larger. Basically, it grows linearly - it goes through a loop `a` times.

#### Russian Peasant's Algorithm (Ancient Egyptian Multiplication)

In [6]:
def russian(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        if x % 2 == 1:
            z = z + y
        y = y << 1
        x = x >> 1
    return z

In [7]:
russian(2, 5)

10

In [8]:
russian(3, 14)

42

In [9]:
17 >> 1

8

In [10]:
16 << 1

32

In [11]:
15 << 1

30

In [12]:
8 >> 1

4

In [13]:
0 << 1

0

Example: `russian(14, 11)`:

    x     y      z
    14    11     0
    7     22     0
    3     44     22
    1     88     66
    0     175    154

#### Why does this work?

We still use $ab = xy + z$ - we'd like to prove this is always true. The base case is true. Now we need the inductive step - does it hold at the top and bottom of the while loop? We need two cases: $x$ is odd and $x$ is even.

If $x$ is odd:
\begin{equation}
\begin{split}
x' =&~ (x - 1)/2 \\
y' =&~ 2y \\
z' =&~ z + y \\
\Rightarrow x'y' + z' =&~ (x-1)/2 \times 2y + z + y \\
=&~ xy - y + z + y \\
=&~ xy + z \\
=&~ ab
\end{split}
\end{equation}

If $x$ is even:
\begin{equation}
\begin{split}
x' =&~ x/2 \\
y' =&~ 2y \\
z' =& z \\
\Rightarrow x'y' + z' =&~ (x/2)(2y) + z \\
=&~ xy + z \\
=&~ ab
\end{split}
\end{equation}

How many additions are there for `russian(20, 7)`?

    x     y     z
    20    7     0
    10    14    0
    5     28    0
    2     56    28
    1     112   28
    0     224   140
    
There are two additions.

Cool trick: there is a relationship between the binary representation of 20 and the number of additions. The binary rep of 20 is `010100`. The addition steps occur when the bit shifts are odd:

    x     y     z    add?
    20    7     0    0
    10    14    0    0
    5     28    0    1
    2     56    28   0
    1     112   28   1
    0     224   140  0
    
Is this commutative? The binary representation of 7 is `111` - so do we have three additions when ordering the computation that way?

    x    y    z     add?
    7    20   0     1
    3    40   20    1
    1    80   60    1
    0    160  140   0

So, yes, there are three additions.

#### `naive` vs `Russian` - which is better?

What matters to us? `naive` is probably more readable. Which is faster? `Russain` is _vastly_ faster for large numbers. 

#### measuring time

Simplifying assumptions

1. simple statements take unit time
2. the time it takes to execute `n` statements in sequence takes `n` units of time (ignore pipelining and caching)
3. therefore the running time for a loop is the time required for the block times the number of loops

In [14]:
# how many units?
s = 0                 # 1
for i in range(10):   # 
    s = s + 1         # 1 + (n = 1) * 10 = 11  
print s               # 12

10


In [15]:
def countdown(x):
    y = 0               # 1
    while x > 0:        # don't count these, 1
        x = x - 5       # 1 + (n * 1)
        y = y + 1       # 1 + (n * 2) -> n = x % 5
    print y             # 2 + (x / 5) * 2
    
print countdown(50)     # 3 + (x / 5) * 2

10
None


In [16]:
def ncomp(n):
    if n % 5 > 0:
        print 3 + ((n / 5) + 1) * 2
    else:
        print 3 + (n / 5) * 2

In [17]:
ncomp(50)
ncomp(5)
ncomp(6)

23
5
7


In [18]:
# instructor solution
import math

def time(n):
    return 3 + 2 * math.ceil(n/5.0)

In [19]:
print time(50)
print time(5)
print time(6)

23.0
5.0
7.0


In [20]:
# count steps in naive
# NOTE: we don't count loop evaluation and we aren't counting the return
def time(a):
    steps = 3
    loopsteps = 2*a
    return steps + loopsteps

In [21]:
print time(4)
print time(6)
print time(8)

11
15
19


In [22]:
# count steps in russian - how many times can we divide x in half before it hits 0
# x      0  1  2  3  4  5  6  7  8  9  10  11  
# #halvs 0  1  2  2  3  3  3  3  4  4  4   4

In [23]:
for x in range(11):
    n = x + 1
    print n, math.floor(math.log(n, 2)) + 1

1 1.0
2 2.0
3 2.0
4 3.0
5 3.0
6 3.0
7 3.0
8 4.0
9 4.0
10 4.0
11 4.0


What about the `x % 2` statement in `russian`? It turns out that it will be executed once for each "on" bit in the binary representation of `a`. It is easy to put an upper bound... and `log(n, 2) + 1` is actually a count of the maximum number of "on" bits! For example, think about 8 (1000) and 7 (111) - `log(7, 2) + 1 = 3`, which is maximum number of "on" bits for anything less than 7.

In [24]:
def time_russ_upper(n):
    """
    upper bound
    """
    steps = 4 * (math.floor(math.log(n, 2)) + 1) + 3

#### Divide and Conquer

Multiplication is repeated addition. We can re-group each multiplication as two different sums of half the numbers - this roughly halves the number of addition operations. We can then also break each of those groups in half and effectively halve the number of additions again, and so on (although, eventually, this seems to lead us back to where we started - the nice thing, I suppose, is that we are always _doubling_ when adding subgroups, and doubling is an easy operation for a computer (bit shift)).

The idea behind _divide and conquer_ is to break a problem into roughly equal sized subproblems, solve them separately, and then combine the results.

In [25]:
def rec_russian(a, b):
    if a == 0:
        return 0
    if a % 2 == 0:
        return 2 * rec_russian(a/2, b)
    return b + 2 * rec_russian((a - 1) / 2, b)

In [26]:
rec_russian(3, 4)

12

In [27]:
%timeit rec_russian(1000,1000)

100000 loops, best of 3: 7.31 µs per loop


#### Recurrence Relation

    T(a) = if a = 0, 1
         = elif a is even, 3 + T(a/2)
         = else            3 + T((a-1)/2)
         
    T(0) = 1
    T(1) = 3            = 4
    T(2) = 3 + T(1)     = 7
    T(3) = 3 + T(1)     = 7
    T(4) = 3 + T(2)     = 10
    T(5) = 3 + T(2)     = 10
    T(6) = 3 + T(3)     = 10
    T(7) = 3 + T(3)     = 10
    T(8) = 3 + T(4)     = 13
         

In [28]:
def rec_cand1(a):
    T_a = math.floor(math.log(a, 2)) + 1
    return T_a

def rec_cand2(a):
    T_a = 3 * math.floor(math.log(a, 2)) + 1
    return T_a

def rec_cand3(a):
    T_a = 3 * math.floor(math.log(a, 2)) + 3
    return T_a

def rec_cand4(a):
    T_a = 3 * math.floor(math.log(a, 2)) + 4
    return T_a

vals = list(range(1, 9))

l1 = [rec_cand1(x) for x in vals]
l2 = [rec_cand2(x) for x in vals]
l3 = [rec_cand3(x) for x in vals]
l4 = [rec_cand4(x) for x in vals]

print(l1)
print(l2)
print(l3)
print(l4)    

[1.0, 2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 4.0]
[1.0, 4.0, 4.0, 7.0, 7.0, 7.0, 7.0, 10.0]
[3.0, 6.0, 6.0, 9.0, 9.0, 9.0, 9.0, 12.0]
[4.0, 7.0, 7.0, 10.0, 10.0, 10.0, 10.0, 13.0]


# Problem Set 1

## Problem 1

    Tyrone  --  Marcie  --  Sayo
    |           |           |
    Hubert  --  Javier  --  Christian

An Eulerian path starting with Javier ends at which node?

Recall an undirected, connected graph has an Eulerian path if it has 0 or 2 nodes of odd degree. Here we have two nodes of odd degree, "Marcie" and "Javier".

Although I can't find a theorem for it in a very brief internet scan, hand-trials seem to indicate if we start with Javier we must end with Marcie. Evidently this was also specified in class, but my notes do not reflect it well: In an undirected, connected graph, all nodes must be of even degree except the start and end node. Here, our start node, Javier, is odd so our end node must also be odd and the only possible choice is Marcie.

Note that simple hand trials seem to confirm the Eulerian path here _must_ start and end with either Javier/Marcie or Marcie/Javier.

## Problem 2

    Tyrone  --  Marcie  --  Sayo
    |           |           |
    Hubert  --  Javier  --  Christian

How many Eulerian paths does the graph have starting from Javier?

There is a general theorem for this, but it is hard to absorb quickly:

https://en.wikipedia.org/wiki/BEST_theorem

Simple counting shows two paths for each outpath from Javier, so 6? This is correct, and the official solution just counted the paths by hand...

## Problem 3

Create tour...

In [29]:
# Eulerian Tour v1
#
# Write a function `create_tour` that takes as input 
# a list of nodes and outputs a graph as a list of
# tuples representing edges between nodes that have
# an Eulerian tour.

def create_tour(nodes):
    """
    our tour must be a list because we must support indexing
    however, there should be no duplicate edges
    
    example solution:
        nodes = [1, 2, 3]
        return [(1, 3), (1, 2), (2, 3)]
        
    TA solution is very elegant (although he never shows `edge()`,
    it must just make tuples)
    --
    tour= []
    l = len(nodes)
    for i in range(l):
        t = edge(nodes[i], nodes[(i+1) % l])
        tour.append(t)
    return tour
    """
    # first, remove duplicate nodes
    nodes = list(set(nodes))   
    tour = []
    nnodes = len(nodes)
    # loop over the nodes and connect them in sequential pairs
    for i in xrange(nnodes - 1):
        j = i + 1
        tour.append((nodes[i], nodes[j]))
    # the loop misses the reconnection from the last node to the first
    tour.append((nodes[0], nodes[-1]))
    # not really needed except to guard against two node sets...
    tour = list(set(tour))
    return tour

def get_degree(tour):
    """
    tour is a list of tuples representing edges between nodes
    
    return a dictionary keyed by node containing the degree of 
    each key
    """
    degree = {}
    for x, y in tour:
        degree[x] = degree.get(x, 0) + 1
        degree[y] = degree.get(y, 0) + 1
    return degree

def check_edge(t, b, nodes):
    """
    t - tuple representing an edge
    b - origin node
    nodes - set of nodes already visited
    
    if we can get to a new node from `b` following `t`, then 
    return the new node, else return `None`
    """
    if t[0] == b:
        if t[1] not in nodes:
            return t[1]
    elif t[1] == b:
        if t[0] not in nodes:
            return t[0]
    return None

def connected_nodes(tour):
    """
    tour - a list of tuples representing all the edges
    
    return the set of nodes reachable from the fist node in `tour`
    """
    a = tour[0][0]      # first node from first tuple in list
    nodes = set([a])    # visited
    explore = set([a])
    while len(explore) > 0:
        # see what other nodes are reachable
        b = explore.pop()
        for t in tour:
            node = check_edge(t, b, nodes)
            if node is None:
                continue
            nodes.add(node)
            explore.add(node)
    return nodes

def is_eulerian_tour(nodes, tour):
    # all nodes must be even degree (here)
    # every node must be in the graph
    degree = get_degree(tour)
    for node in nodes:
        try:
            d = degree[node]
            if d % 2 == 1:
                print "Node %s has odd degree" % node
                return False
        except KeyError:
            print "Node %s was not in your tour" % node
            return False
    connected = connected_nodes(tour)
    if len(connected) == len(nodes):
        return True
    else:
        print "Your graph was't connected"
        return False
    
def test():
    nodes = [20, 21, 22, 23, 24, 25]
    tour = create_tour(nodes)
    return is_eulerian_tour(nodes, tour)

In [30]:
# nodes = [20, 21, 22, 23, 24, 25]
nodes = [20, 22, 24]
tour = create_tour(nodes)
print tour
degree = get_degree(tour)
print degree
test()

[(24, 22), (20, 22), (24, 20)]
{24: 2, 20: 2, 22: 2}


True

## Problem 4

What are the advantages and disadvantages of representing a graph as a list of tuples?

Advantages:

* iterable
* simple

Disadvantages:

* no enforcement against duplicates (maybe you want duplicates though)
* not clear how to represent isolated nodes
* no natural enforcement mechanism to keep the graph connected (maybe you want unconnected graphs though)

## Problem 5

Recall from lecture:

In [31]:
def naive(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        z += y
        x -= 1
    return z

Suppose we are computing `naive(63, 12)`. At some point we have `x = 20` and `y = 12`. What is `z`?

Recall we proved `ab = xy + z` always, so `z = ab - xy`. At this point...

In [32]:
63 * 12 - 20 * 12

516

... `z` is 516.

## Problem 6

Here is a recursive multiplication algorithm:

In [33]:
def rec_naive(a, b):
    if a == 0:
        return 0
    return b + rec_naive(a - 1, b)

How many additions are required to compute `rec_naive(17, 6)` = 102?

Here, we go through the loop 17 times. (We should at least do `rec_naive(6, 17)`, haha.)

## Problem 7

Recall the `russian` multiplication algorithm from lecture:

In [34]:
def russian(a, b):
    x = a
    y = b
    z = 0
    while x > 0:
        if x % 2 == 1:
            z = z + y
        y = y << 1
        x = x >> 1
        print(x, y, z)  # just to check...
    return z

Suppose we are computing `russian(63,12)`. At some point we have `x=7` and `z=84`. What is `y` at that moment?

We proved `ab = xy + z` always. Therefore, `y = (ab - z)/x`, or:

In [35]:
(63*12 - 84)/7

96

In [36]:
russian(63, 12)

(31, 24, 12)
(15, 48, 36)
(7, 96, 84)
(3, 192, 180)
(1, 384, 372)
(0, 768, 756)


756

## Problem 8

In [37]:
def clique(n):
    print "in a clique..."
    for j in range(n):
        print "range eval outer"
        for i in range(j):
            print "range eval inner"
            print i, "is friends with", j

In [38]:
clique(4)

in a clique...
range eval outer
range eval outer
range eval inner
0 is friends with 1
range eval outer
range eval inner
0 is friends with 2
range eval inner
1 is friends with 2
range eval outer
range eval inner
0 is friends with 3
range eval inner
1 is friends with 3
range eval inner
2 is friends with 3


How many instructions are required for `clique(4)`? Count each `print` as one unit and each `range` evaluation as 1 unit.

There are seven `print` statements. `range` is evaluated four times in the outer loop, and then 0, 1, 2, and 3 times on the inner loops. Therefore, the total is 17.

End of range too I guess means...

There are seven `print` statements. `range` is evaluated 4+1 times in the outer loop, and then 0+1, 1+1, 2+1, and 3+1 times on the inner loops. Therefore, the total is 22.

Their answer is _12_.

They come up with this by counting statements like `range(4)` as requiring _one evaluation_ and not as requiring _four evaluations_. Nobody in the forums complained about this, but I would certainly say that resolving `range(4)` would require four instructions, not one. 

In [39]:
scipy.special.binom(4, 2)

6.0

## Problem 9

Write a function evaluate how many evaluations `click(n)` requires (presumably under their strange rules).

In [40]:
def count(n):
    nprints = 1 + scipy.special.binom(n, 2)
    nranges = 1                               # outer
    nranges += n                              # inner
    return nprints + nranges

In [41]:
count(4)

12.0

In [42]:
# w/o scipy
def count(n):
    if n == 1:
        return 3
    
    def factorial(n):
        if n == 0:
            return 1
        return n * factorial(n-1)
    
    nprints = 1 + factorial(n) / (factorial(n-2) * 2)
    nranges = 1 + n
    return nprints + nranges

In [43]:
count(4)

12

In [44]:
count(3)

8

In [45]:
# re-def ...
def clique(n):
    print "in a clique..."
    print "range eval outer"
    for j in range(n):
        print "range eval inner"
        for i in range(j):
            print i, "is friends with", j

In [46]:
clique(4)

in a clique...
range eval outer
range eval inner
range eval inner
0 is friends with 1
range eval inner
0 is friends with 2
1 is friends with 2
range eval inner
0 is friends with 3
1 is friends with 3
2 is friends with 3


In [47]:
# re-def again ...
def clique(n):
    l = ["in a clique..."]
    l.extend(["range eval outer"])
    for j in range(n):
        l.extend(["range eval inner"])
        for i in range(j):
            s = i, "is friends with", j
            l.extend([s])
    return l

In [48]:
def tester(n):
    ss = clique(n)
    # print ss
    print len(ss)
    print count(n)

In [49]:
tester(4)

12
12


In [50]:
# but, wrong?...
def course_count(n):
    c = 2 + n + n * (n - 1) / 2
    return c

In [51]:
def course_tester(n):
    ss = clique(n)
    # print ss
    print len(ss)
    print course_count(n)

In [52]:
course_tester(4)

12
12


In [53]:
for i in range(2, 7):
    tester(i)
    course_tester(i)

5
5
5
5
8
8
8
8
12
12
12
12
17
17
17
17
23
23
23
23


In [54]:
ll = []
for i in range(2, 100):
    ll.append(count(n) == course_count(n))
    
print all(ll)

True


So, my solution was correct, but it somehow could not be defined inside their Python, even though it runs here. Theirs is much more elegant, either way

## Problem 10

Find an Eulerian tour for a graph represented as a list of tuples.

In [55]:
def get_degree_dict(tour):
    """
    tour is a list of tuples representing edges between nodes
    
    return a dictionary keyed by node containing the degree of 
    each key
    """
    degree = {}
    for x, y in tour:
        degree[x] = degree.get(x, 0) + 1
        degree[y] = degree.get(y, 0) + 1
    return degree

def get_degree_from_dict(degree):
    max_d = 0
    for k, v in degree.iteritems():
        if v > max_d:
            max_d = v
    return max_d

def get_degree_from_tour(tour):
    degdict = get_degree_dict(tour)
    max_d = get_degree_from_dict(degdict)
    return max_d

def count_odd_degree_nodes(tour):
    degdict = get_degree_dict(tour)
    num_odd = 0
    for k, v in degdict.iteritems():
        if v % 2 == 1:
            num_odd += 1
    return num_odd

In [71]:
# Find Eulerian Tour
#
# Write a function that takes a graph, represented as a list of tuples,
# and returns a list of nodes that you would follow on an Eulerian tour.
# 
# e.g., for [(1,2), (2,3), (3,1)]
# a possible tour is [1, 2, 3, 1]

def find_eulerian_tour(graph):
    
    if len(graph) == 0:
        return []
    
    gg = graph[::]
    print gg
    
    degree_dict = {}
    for x, y in gg:
        degree_dict[x] = degree_dict.get(x, 0) + 1
        degree_dict[y] = degree_dict.get(y, 0) + 1
    print(degree_dict)
        
    odds = []
    for k, v in degree_dict.iteritems():
        if v % 2 == 1:
            odds.append(k)
    num_odd = len(odds)
    print(odds)
        
    nodes = []
    if num_odd == 0:
        edge = gg.pop(0)
        nodes.append(edge[0])
        nodes.append(edge[1])
    elif num_odd == 2:
        first_odd_edge = None
        for edge in gg:
            if edge[0] in odds:
                first_odd_index = gg.index(edge)
                first_odd_edge = gg.pop(first_odd_index)
                break
            elif edge[1] in odds:
                first_odd_index = gg.index(edge)
                first_odd_edge = gg.pop(first_odd_index)
                first_odd_edge = first_odd_edge[1], first_odd_edge[0]
                break
        nodes.append(first_odd_edge[0])
        nodes.append(first_odd_edge[1])
    else:
        return nodes
    
    print "Starting steps: ", nodes
    print ""
    
    while len(gg) > 0:
        possible_next_steps = []
        
        for edge in gg:
            if edge[0] == nodes[-1] or edge[1] == nodes[-1]:
                possible_next_steps.append(edge)
                
        # connected check
        connected = False
        for edge in gg:
            for node in nodes:
                if edge[0] == node or edge[1] == node:
                    connected = True
                    break
        if not connected:
            print "Not connected!"
            return nodes
                 
        back_steps = []
        while len(possible_next_steps) == 0:
            print "Backing off..."
            print nodes
            print gg
            edge = nodes.pop(-1), nodes[-1]
            back_steps.append(edge)
            print "Checking for possible paths..."
            for edge in gg:
                if edge[0] == nodes[-1] or edge[1] == nodes[-1]:
                    possible_next_steps.append(edge)

        while len(back_steps) > 0:
            edge = back_steps.pop()
            gg.append(edge)
        
        print "Possible next steps...", possible_next_steps
        print "Nodes: ", nodes
        print "Graph: ", gg
        
        next_edge_index = gg.index(possible_next_steps[0])
        edge = gg.pop(next_edge_index)
        print "Next edge: ", edge
        if edge[0] == nodes[-1]:
            nodes.append(edge[1])
        else:
            nodes.append(edge[0])
        print "New tour: ", nodes
        
        print ""
             
    return nodes

In [70]:
graph = [(0, 1), (1, 5), (1, 7), (4, 5), (4, 8), (1, 6), (3, 7), 
         (5, 9), (2, 4), (0, 4), (2, 5), (3, 6), (8, 9)]
nodes = find_eulerian_tour(graph)
print(nodes)

[(0, 1), (1, 5), (1, 7), (4, 5), (4, 8), (1, 6), (3, 7), (5, 9), (2, 4), (0, 4), (2, 5), (3, 6), (8, 9)]
{0: 2, 1: 4, 2: 2, 3: 2, 4: 4, 5: 4, 6: 2, 7: 2, 8: 2, 9: 2}
[]
Starting steps:  [0, 1]

Possible next steps... [(1, 5), (1, 7), (1, 6)]
Nodes:  [0, 1]
Graph:  [(1, 5), (1, 7), (4, 5), (4, 8), (1, 6), (3, 7), (5, 9), (2, 4), (0, 4), (2, 5), (3, 6), (8, 9)]
Next edge:  (1, 5)
New tour:  [0, 1, 5]

Possible next steps... [(4, 5), (5, 9), (2, 5)]
Nodes:  [0, 1, 5]
Graph:  [(1, 7), (4, 5), (4, 8), (1, 6), (3, 7), (5, 9), (2, 4), (0, 4), (2, 5), (3, 6), (8, 9)]
Next edge:  (4, 5)
New tour:  [0, 1, 5, 4]

Possible next steps... [(4, 8), (2, 4), (0, 4)]
Nodes:  [0, 1, 5, 4]
Graph:  [(1, 7), (4, 8), (1, 6), (3, 7), (5, 9), (2, 4), (0, 4), (2, 5), (3, 6), (8, 9)]
Next edge:  (4, 8)
New tour:  [0, 1, 5, 4, 8]

Possible next steps... [(8, 9)]
Nodes:  [0, 1, 5, 4, 8]
Graph:  [(1, 7), (1, 6), (3, 7), (5, 9), (2, 4), (0, 4), (2, 5), (3, 6), (8, 9)]
Next edge:  (8, 9)
New tour:  [0, 1, 5, 4, 8, 9]

----  
  
**`FAILURE`**`: Test case input: [(1, 2), (2, 3), (3, 1)].`  

				Expected result: ...  


**`FAILURE`**`: Test case input: [(0, 1), (1, 5), (1, 7), (4, 5),
(4, 8), (1, 6), (3, 7), (5, 9),
(2, 4), (0, 4), (2, 5), (3, 6), (8, 9)].`  

				Expected result: ...  


**`FAILURE`**`: Test case input: [(1, 13), (1, 6), (6, 11), (3, 13),
(8, 13), (0, 6), (8, 9),(5, 9), (2, 6), (6, 10), (7, 9),
(1, 12), (4, 12), (5, 14), (0, 1),  (2, 3), (4, 11), (6, 9),
(7, 14),  (10, 13)].`  

				Expected result: ...  


**`FAILURE`**`: Test case input: [(8, 16), (8, 18), (16, 17), (18, 19),
(3, 17), (13, 17), (5, 13),(3, 4), (0, 18), (3, 14), (11, 14),
(1, 8), (1, 9), (4, 12), (2, 19),(1, 10), (7, 9), (13, 15),
(6, 12), (0, 1), (2, 11), (3, 18), (5, 6), (7, 15), (8, 13), (10, 17)].`  

				Expected result: ...  

------------------  
**`You passed 0 out of 4 test cases`**  

In [57]:
graph = [(1, 2), (2, 3), (3, 1)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 3, 1]
print nodes == expected_tour

[(1, 2), (2, 3), (3, 1)]
{1: 2, 2: 2, 3: 2}
[]
Starting steps:  [1, 2]

Possible next steps... [(2, 3)]
Nodes:  [1, 2]
Graph:  [(2, 3), (3, 1)]
Next edge:  (2, 3)
New tour:  [1, 2, 3]

Possible next steps... [(3, 1)]
Nodes:  [1, 2, 3]
Graph:  [(3, 1)]
Next edge:  (3, 1)
New tour:  [1, 2, 3, 1]

[1, 2, 3, 1]
True


In [58]:
graph = [(1, 2), (3, 1), (2, 3)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 3, 1]
print nodes == expected_tour

[(1, 2), (3, 1), (2, 3)]
{1: 2, 2: 2, 3: 2}
[]
Starting steps:  [1, 2]

Possible next steps... [(2, 3)]
Nodes:  [1, 2]
Graph:  [(3, 1), (2, 3)]
Next edge:  (2, 3)
New tour:  [1, 2, 3]

Possible next steps... [(3, 1)]
Nodes:  [1, 2, 3]
Graph:  [(3, 1)]
Next edge:  (3, 1)
New tour:  [1, 2, 3, 1]

[1, 2, 3, 1]
True


In [59]:
graph = [(1, 2), (1, 6), (6, 5), (2, 5), (5, 4), (2, 3), (3, 4)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [2, 1, 6, 5, 2, 3, 4, 5]
print nodes == expected_tour

[(1, 2), (1, 6), (6, 5), (2, 5), (5, 4), (2, 3), (3, 4)]
{1: 2, 2: 3, 3: 2, 4: 2, 5: 3, 6: 2}
[2, 5]
Starting steps:  [2, 1]

Possible next steps... [(1, 6)]
Nodes:  [2, 1]
Graph:  [(1, 6), (6, 5), (2, 5), (5, 4), (2, 3), (3, 4)]
Next edge:  (1, 6)
New tour:  [2, 1, 6]

Possible next steps... [(6, 5)]
Nodes:  [2, 1, 6]
Graph:  [(6, 5), (2, 5), (5, 4), (2, 3), (3, 4)]
Next edge:  (6, 5)
New tour:  [2, 1, 6, 5]

Possible next steps... [(2, 5), (5, 4)]
Nodes:  [2, 1, 6, 5]
Graph:  [(2, 5), (5, 4), (2, 3), (3, 4)]
Next edge:  (2, 5)
New tour:  [2, 1, 6, 5, 2]

Possible next steps... [(2, 3)]
Nodes:  [2, 1, 6, 5, 2]
Graph:  [(5, 4), (2, 3), (3, 4)]
Next edge:  (2, 3)
New tour:  [2, 1, 6, 5, 2, 3]

Possible next steps... [(3, 4)]
Nodes:  [2, 1, 6, 5, 2, 3]
Graph:  [(5, 4), (3, 4)]
Next edge:  (3, 4)
New tour:  [2, 1, 6, 5, 2, 3, 4]

Possible next steps... [(5, 4)]
Nodes:  [2, 1, 6, 5, 2, 3, 4]
Graph:  [(5, 4)]
Next edge:  (5, 4)
New tour:  [2, 1, 6, 5, 2, 3, 4, 5]

[2, 1, 6, 5, 2, 3, 4, 5]
T

In [60]:
graph = [(1, 2), (2, 3), (3, 1), (1, 5), (5, 2), (5, 3), (3, 4), (4, 2)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 3, 1, 5, 2, 4, 3, 5]
print nodes == expected_tour

[(1, 2), (2, 3), (3, 1), (1, 5), (5, 2), (5, 3), (3, 4), (4, 2)]
{1: 3, 2: 4, 3: 4, 4: 2, 5: 3}
[1, 5]
Starting steps:  [1, 2]

Possible next steps... [(2, 3), (5, 2), (4, 2)]
Nodes:  [1, 2]
Graph:  [(2, 3), (3, 1), (1, 5), (5, 2), (5, 3), (3, 4), (4, 2)]
Next edge:  (2, 3)
New tour:  [1, 2, 3]

Possible next steps... [(3, 1), (5, 3), (3, 4)]
Nodes:  [1, 2, 3]
Graph:  [(3, 1), (1, 5), (5, 2), (5, 3), (3, 4), (4, 2)]
Next edge:  (3, 1)
New tour:  [1, 2, 3, 1]

Possible next steps... [(1, 5)]
Nodes:  [1, 2, 3, 1]
Graph:  [(1, 5), (5, 2), (5, 3), (3, 4), (4, 2)]
Next edge:  (1, 5)
New tour:  [1, 2, 3, 1, 5]

Possible next steps... [(5, 2), (5, 3)]
Nodes:  [1, 2, 3, 1, 5]
Graph:  [(5, 2), (5, 3), (3, 4), (4, 2)]
Next edge:  (5, 2)
New tour:  [1, 2, 3, 1, 5, 2]

Possible next steps... [(4, 2)]
Nodes:  [1, 2, 3, 1, 5, 2]
Graph:  [(5, 3), (3, 4), (4, 2)]
Next edge:  (4, 2)
New tour:  [1, 2, 3, 1, 5, 2, 4]

Possible next steps... [(3, 4)]
Nodes:  [1, 2, 3, 1, 5, 2, 4]
Graph:  [(5, 3), (3, 4)]


In [61]:
graph = [(1, 2), (3, 2), (3, 4), (5, 4), (5, 6), (6, 7), (1, 7)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 3, 4, 5, 6, 7, 1]
print nodes == expected_tour

[(1, 2), (3, 2), (3, 4), (5, 4), (5, 6), (6, 7), (1, 7)]
{1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2}
[]
Starting steps:  [1, 2]

Possible next steps... [(3, 2)]
Nodes:  [1, 2]
Graph:  [(3, 2), (3, 4), (5, 4), (5, 6), (6, 7), (1, 7)]
Next edge:  (3, 2)
New tour:  [1, 2, 3]

Possible next steps... [(3, 4)]
Nodes:  [1, 2, 3]
Graph:  [(3, 4), (5, 4), (5, 6), (6, 7), (1, 7)]
Next edge:  (3, 4)
New tour:  [1, 2, 3, 4]

Possible next steps... [(5, 4)]
Nodes:  [1, 2, 3, 4]
Graph:  [(5, 4), (5, 6), (6, 7), (1, 7)]
Next edge:  (5, 4)
New tour:  [1, 2, 3, 4, 5]

Possible next steps... [(5, 6)]
Nodes:  [1, 2, 3, 4, 5]
Graph:  [(5, 6), (6, 7), (1, 7)]
Next edge:  (5, 6)
New tour:  [1, 2, 3, 4, 5, 6]

Possible next steps... [(6, 7)]
Nodes:  [1, 2, 3, 4, 5, 6]
Graph:  [(6, 7), (1, 7)]
Next edge:  (6, 7)
New tour:  [1, 2, 3, 4, 5, 6, 7]

Possible next steps... [(1, 7)]
Nodes:  [1, 2, 3, 4, 5, 6, 7]
Graph:  [(1, 7)]
Next edge:  (1, 7)
New tour:  [1, 2, 3, 4, 5, 6, 7, 1]

[1, 2, 3, 4, 5, 6, 7, 1]
True


In [62]:
graph = [(1, 2), (2, 4), (2, 5), (2, 3), (3, 5), (3, 4), (3, 1), (4, 5), (4, 1), (1, 5)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 4, 3, 2, 5, 3, 1, 4, 5, 1]
print nodes == expected_tour

[(1, 2), (2, 4), (2, 5), (2, 3), (3, 5), (3, 4), (3, 1), (4, 5), (4, 1), (1, 5)]
{1: 4, 2: 4, 3: 4, 4: 4, 5: 4}
[]
Starting steps:  [1, 2]

Possible next steps... [(2, 4), (2, 5), (2, 3)]
Nodes:  [1, 2]
Graph:  [(2, 4), (2, 5), (2, 3), (3, 5), (3, 4), (3, 1), (4, 5), (4, 1), (1, 5)]
Next edge:  (2, 4)
New tour:  [1, 2, 4]

Possible next steps... [(3, 4), (4, 5), (4, 1)]
Nodes:  [1, 2, 4]
Graph:  [(2, 5), (2, 3), (3, 5), (3, 4), (3, 1), (4, 5), (4, 1), (1, 5)]
Next edge:  (3, 4)
New tour:  [1, 2, 4, 3]

Possible next steps... [(2, 3), (3, 5), (3, 1)]
Nodes:  [1, 2, 4, 3]
Graph:  [(2, 5), (2, 3), (3, 5), (3, 1), (4, 5), (4, 1), (1, 5)]
Next edge:  (2, 3)
New tour:  [1, 2, 4, 3, 2]

Possible next steps... [(2, 5)]
Nodes:  [1, 2, 4, 3, 2]
Graph:  [(2, 5), (3, 5), (3, 1), (4, 5), (4, 1), (1, 5)]
Next edge:  (2, 5)
New tour:  [1, 2, 4, 3, 2, 5]

Possible next steps... [(3, 5), (4, 5), (1, 5)]
Nodes:  [1, 2, 4, 3, 2, 5]
Graph:  [(3, 5), (3, 1), (4, 5), (4, 1), (1, 5)]
Next edge:  (3, 5)
New t

In [63]:
graph = []
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = []
print nodes == expected_tour

[]
True


In [64]:
graph = [(1, 2)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2]
print nodes == expected_tour

[(1, 2)]
{1: 1, 2: 1}
[1, 2]
Starting steps:  [1, 2]

[1, 2]
True


In [65]:
graph = [(1, 2), (2, 3), (3, 4), (4, 1), (1, 5), (5, 3)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 3, 4, 1, 5, 3]
print nodes == expected_tour

[(1, 2), (2, 3), (3, 4), (4, 1), (1, 5), (5, 3)]
{1: 3, 2: 2, 3: 3, 4: 2, 5: 2}
[1, 3]
Starting steps:  [1, 2]

Possible next steps... [(2, 3)]
Nodes:  [1, 2]
Graph:  [(2, 3), (3, 4), (4, 1), (1, 5), (5, 3)]
Next edge:  (2, 3)
New tour:  [1, 2, 3]

Possible next steps... [(3, 4), (5, 3)]
Nodes:  [1, 2, 3]
Graph:  [(3, 4), (4, 1), (1, 5), (5, 3)]
Next edge:  (3, 4)
New tour:  [1, 2, 3, 4]

Possible next steps... [(4, 1)]
Nodes:  [1, 2, 3, 4]
Graph:  [(4, 1), (1, 5), (5, 3)]
Next edge:  (4, 1)
New tour:  [1, 2, 3, 4, 1]

Possible next steps... [(1, 5)]
Nodes:  [1, 2, 3, 4, 1]
Graph:  [(1, 5), (5, 3)]
Next edge:  (1, 5)
New tour:  [1, 2, 3, 4, 1, 5]

Possible next steps... [(5, 3)]
Nodes:  [1, 2, 3, 4, 1, 5]
Graph:  [(5, 3)]
Next edge:  (5, 3)
New tour:  [1, 2, 3, 4, 1, 5, 3]

[1, 2, 3, 4, 1, 5, 3]
True


In [66]:
graph = [(1, 1)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 1]
print nodes == expected_tour

[(1, 1)]
{1: 2}
[]
Starting steps:  [1, 1]

[1, 1]
True


In [67]:
graph = [(1, 2), (2, 3), (3, 1), (3, 4), (4, 3)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 3, 4, 3, 1]
print nodes == expected_tour

[(1, 2), (2, 3), (3, 1), (3, 4), (4, 3)]
{1: 2, 2: 2, 3: 4, 4: 2}
[]
Starting steps:  [1, 2]

Possible next steps... [(2, 3)]
Nodes:  [1, 2]
Graph:  [(2, 3), (3, 1), (3, 4), (4, 3)]
Next edge:  (2, 3)
New tour:  [1, 2, 3]

Possible next steps... [(3, 1), (3, 4), (4, 3)]
Nodes:  [1, 2, 3]
Graph:  [(3, 1), (3, 4), (4, 3)]
Next edge:  (3, 1)
New tour:  [1, 2, 3, 1]

Backing off...
[1, 2, 3, 1]
[(3, 4), (4, 3)]
Checking for possible paths...
Possible next steps... [(3, 4), (4, 3)]
Nodes:  [1, 2, 3]
Graph:  [(3, 4), (4, 3), (1, 3)]
Next edge:  (3, 4)
New tour:  [1, 2, 3, 4]

Possible next steps... [(4, 3)]
Nodes:  [1, 2, 3, 4]
Graph:  [(4, 3), (1, 3)]
Next edge:  (4, 3)
New tour:  [1, 2, 3, 4, 3]

Possible next steps... [(1, 3)]
Nodes:  [1, 2, 3, 4, 3]
Graph:  [(1, 3)]
Next edge:  (1, 3)
New tour:  [1, 2, 3, 4, 3, 1]

[1, 2, 3, 4, 3, 1]
True


In [68]:
graph = [(1, 2), (1, 3), (2, 3), (2, 4), (2, 6), (3, 4), (3, 5), (4, 5), (4, 6), (9, 10), (10, 11), (11, 9)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [1, 2, 3, 4, 2, 6, 4, 5, 3, 1]
print nodes == expected_tour

[(1, 2), (1, 3), (2, 3), (2, 4), (2, 6), (3, 4), (3, 5), (4, 5), (4, 6), (9, 10), (10, 11), (11, 9)]
{1: 2, 2: 4, 3: 4, 4: 4, 5: 2, 6: 2, 9: 2, 10: 2, 11: 2}
[]
Starting steps:  [1, 2]

Possible next steps... [(2, 3), (2, 4), (2, 6)]
Nodes:  [1, 2]
Graph:  [(1, 3), (2, 3), (2, 4), (2, 6), (3, 4), (3, 5), (4, 5), (4, 6), (9, 10), (10, 11), (11, 9)]
Next edge:  (2, 3)
New tour:  [1, 2, 3]

Possible next steps... [(1, 3), (3, 4), (3, 5)]
Nodes:  [1, 2, 3]
Graph:  [(1, 3), (2, 4), (2, 6), (3, 4), (3, 5), (4, 5), (4, 6), (9, 10), (10, 11), (11, 9)]
Next edge:  (1, 3)
New tour:  [1, 2, 3, 1]

Backing off...
[1, 2, 3, 1]
[(2, 4), (2, 6), (3, 4), (3, 5), (4, 5), (4, 6), (9, 10), (10, 11), (11, 9)]
Checking for possible paths...
Possible next steps... [(3, 4), (3, 5)]
Nodes:  [1, 2, 3]
Graph:  [(2, 4), (2, 6), (3, 4), (3, 5), (4, 5), (4, 6), (9, 10), (10, 11), (11, 9), (1, 3)]
Next edge:  (3, 4)
New tour:  [1, 2, 3, 4]

Possible next steps... [(2, 4), (4, 5), (4, 6)]
Nodes:  [1, 2, 3, 4]
Graph:

In [69]:
graph = [(1, 2), (2, 2), (2, 3), (3, 4), (4, 3), (4, 5), (1, 4), (5, 2)]
nodes = find_eulerian_tour(graph)
print(nodes)
expected_tour = [2, 1, 4, 3, 2, 2, 5, 4, 3]
print nodes == expected_tour

[(1, 2), (2, 2), (2, 3), (3, 4), (4, 3), (4, 5), (1, 4), (5, 2)]
{1: 2, 2: 5, 3: 3, 4: 4, 5: 2}
[2, 3]
Starting steps:  [2, 1]

Possible next steps... [(1, 4)]
Nodes:  [2, 1]
Graph:  [(2, 2), (2, 3), (3, 4), (4, 3), (4, 5), (1, 4), (5, 2)]
Next edge:  (1, 4)
New tour:  [2, 1, 4]

Possible next steps... [(3, 4), (4, 3), (4, 5)]
Nodes:  [2, 1, 4]
Graph:  [(2, 2), (2, 3), (3, 4), (4, 3), (4, 5), (5, 2)]
Next edge:  (3, 4)
New tour:  [2, 1, 4, 3]

Possible next steps... [(2, 3), (4, 3)]
Nodes:  [2, 1, 4, 3]
Graph:  [(2, 2), (2, 3), (4, 3), (4, 5), (5, 2)]
Next edge:  (2, 3)
New tour:  [2, 1, 4, 3, 2]

Possible next steps... [(2, 2), (5, 2)]
Nodes:  [2, 1, 4, 3, 2]
Graph:  [(2, 2), (4, 3), (4, 5), (5, 2)]
Next edge:  (2, 2)
New tour:  [2, 1, 4, 3, 2, 2]

Possible next steps... [(5, 2)]
Nodes:  [2, 1, 4, 3, 2, 2]
Graph:  [(4, 3), (4, 5), (5, 2)]
Next edge:  (5, 2)
New tour:  [2, 1, 4, 3, 2, 2, 5]

Possible next steps... [(4, 5)]
Nodes:  [2, 1, 4, 3, 2, 2, 5]
Graph:  [(4, 3), (4, 5)]
Next edg